# Examples on How to Compute Alignments and Skip Alignments

In [1]:
%load_ext autoreload
%autoreload 2
import pm4py
from processtree import *
from alignment import *
import pandas as pd
from tqdm import tqdm
from alignall import *
import statistics
import random
from tqdm import tqdm

In [2]:
# What example do you want to run?
example = 1

## Example 1 (No Cycles, Example from Paper)

In [ ]:
tau = Tau(None, 'Tau', 0)

a2 = Activity(None, 'a', 100000)
b2 = Activity(None, 'b', 100000)
d2 = Activity(None, 'd', 100000)
e2 = Activity(None, 'e', 100000)
f2 = Activity(None, 'f', 100000)
g2 = Activity(None, 'g', 100000)
h2 = Activity(None, 'h', 100000)

xorpre = Xor(None, [tau, a2])
tau.set_parent(xorpre)
a2.set_parent(xorpre)

xormid = Xor(None, [d2, e2])
d2.set_parent(xormid)
e2.set_parent(xormid)

andpost = And(None, [f2, g2, h2])
f2.set_parent(andpost)
g2.set_parent(andpost)
h2.set_parent(andpost)

tree2 = Sequence(None, [xorpre, b2, xormid, andpost])
xorpre.set_parent(tree2)
b2.set_parent(tree2)
xormid.set_parent(tree2)
andpost.set_parent(tree2)
tree2

In [ ]:
process_tree_rf2 = tree2.to_pm4py()
tau_loops_rf2, process_tree_rf2 = insert_cycle_checks(process_tree_rf2)
pm4py.view_process_tree(process_tree_rf2, format='png')
sublog_pt_rf_net2, sublog_pt_rf_init2, sublog_pt_rf_final2 = pm4py.convert.convert_to_petri_net(process_tree_rf2)

## Example 2 (Loops With Cycles)

In [ ]:
tau1 = Tau(None, 'Tau1', 0)
tau2 = Tau(None, 'Tau2', 0)
tau3 = Tau(None, 'Tau3', 0)
tau4 = Tau(None, 'Tau4', 0)

a = Activity(None, 'a', 100000)
b = Activity(None, 'b', 100000)

xor1 = Xor(None, [tau1, a])
tau1.set_parent(xor1)
a.set_parent(xor1)

xor2 = Xor(None, [b, tau4])
b.set_parent(xor2)
tau4.set_parent(xor2)

loop1 = Loop(None, [xor1, tau2])
loop1.id = "LOOP1"
xor1.set_parent(loop1)
tau2.set_parent(loop1)

loop2 = Loop(None, [tau3, xor2])
loop2.id = "LOOP2"
tau3.set_parent(loop2)
xor2.set_parent(loop2)

tree = Sequence(None, [loop1, loop2])
loop1.set_parent(tree)
loop2.set_parent(tree)
tree

In [ ]:
process_tree_rf = tree.to_pm4py()
tau_loops_rf, process_tree_rf = insert_cycle_checks(process_tree_rf)
pm4py.view_process_tree(process_tree_rf, format='png')
sublog_pt_rf_net, sublog_pt_rf_init, sublog_pt_rf_final = pm4py.convert.convert_to_petri_net(process_tree_rf)

In [7]:
if example == 1:
    sublog_pt_net = sublog_pt_rf_net2
    sublog_pt_init = sublog_pt_rf_init2
    sublog_pt_final = sublog_pt_rf_final2
    loop_ids = tau_loops_rf2
    pt = tree2
    var = ['a', 'c', 'g']
else:
    sublog_pt_net = sublog_pt_rf_net
    sublog_pt_init = sublog_pt_rf_init
    sublog_pt_final = sublog_pt_rf_final
    loop_ids = tau_loops_rf
    pt = tree
    var = ['a', 'b']

## Compute All Optimal Alignments

In [ ]:
pm4py_dict = {}
pm4py_times = {}
pm4py_times_first = {}

var_df = pd.DataFrame({'case:concept:name': '1', 'concept:name': var, 'time:timestamp': [pd.Timestamp(year=1000+i, month=1, day=1) for i, _ in enumerate(var)]})
pm4py_res = align_pn_all_multi(var_df, sublog_pt_net, sublog_pt_init, sublog_pt_final, loop_ids, pt, timeout=3600)

for k,(process_time, (agns, has_timed_out, time_first_agn)) in pm4py_res.items():
    pm4py_dict[k] = list(set([tuple([t for t in agns[i]['alignment'] if t.label[1] is not None and not t.label[1].startswith("TAU_entry") and not t.label[1].startswith("TAU_exit")]) for i in range(len(agns))]))
    pm4py_times[k] = -1 if has_timed_out == -1 or process_time > 36*10**11 else process_time
    pm4py_times_first[k] = time_first_agn
print("Number of computed alignments:", len(pm4py_dict[', '.join(var)]))
print("They are:")
pm4py_dict[', '.join(var)]

## Compute All Optimal Skip Alignments in Normal Form

In [ ]:
skip_dict = {}
skip_times = {}
Aligner.set_level_incentive(0)
variant_strings = [var]
futures = align_sk_all(variant_strings, tree2, timeout=3600)
for index, variant in enumerate(futures):
    agns, t = futures[index].result()
    skip_dict[", ".join(variant_strings[index])] = agns
    skip_times[", ".join(variant_strings[index])] = t
print("Number of computed skip alignments in normal form:", len(skip_dict[', '.join(variant_strings[0])]))
print("They are:")
[s.path for s in skip_dict[', '.join(var)]]